In [13]:
import numpy as np
import os # will use later on for getting the datasaet as well as saving the model and output. 
import random

import tensorflow

from keras.layers import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave

In [14]:
# Get images
X = []
number_of_files = 0
for filename in os.listdir('Train'):
    temporary_img = load_img('Train/'+filename)
    temporary_img = img_to_array(temporary_img)
    X.append(temporary_img) #add the image to the array.
    number_of_files += 1 # count the number of files 
# Convert standard array to numpy array for future
X = np.array(X)#, dtype=float) #float gives error

# Set up train and test data
split = int(0.95 * len(X) )
Xtrain = X[:split]
Xtrain = 1.0/255 * Xtrain# divide the numbers in the array by 255 but keep them as float numbers (with 1.0)

In [15]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same')) #this returns values in range of -1 to 1
model.add(UpSampling2D((2, 2)))

In [16]:
model.compile(optimizer='rmsprop', loss='mse')

In [17]:
# Image transformer
# rotate, flip, zoom in on pictures and etc so that the training set becomes larger
# improves the accurracy too!
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

In [18]:
# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    #for every picture get its L and ab channels and save them for later
    for batch in datagen.flow(Xtrain):#, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0] # get the lightness channel
        Y_batch = lab_batch[:,:,:,1:] / 128
        #yield is like return but it just keeps the variables in the memory, that way it doesn't stop the for loop on 1st loop.
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [22]:
print(number_of_files)
print(Xtrain.shape) #number of files to train on, the rest is left for testing and evaluating.

9294
(8829, 256, 256, 3)


In [ ]:
# Train model
tensorboard = TensorBoard(log_dir="output/first_run")
#image_a_b_gen(batch_size) # X elements= 827 # Y elements =  1654
tensorboard = [tensorboard]
#model.fit(x=X,y=Y,batch_size=20,epochs=1)
model.fit_generator(image_a_b_gen(batch_size),
                   steps_per_epoch=8829,
                   epochs=1)#,use_multiprocessing=True)#, callbacks=tensorboard

Epoch 1/1
  36/8829 [..............................] - ETA: 30:54:26 - loss: 0.1833

In [8]:
# Save model
model_json = model.to_json()
with open("model100.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model100.h5")

In [9]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

1/1 [==============================] - 0s 119ms/step
0.0036522254813462496


In [10]:
color_me = []
for filename in os.listdir('Test/'):
    color_me.append(img_to_array(load_img('Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [11]:
# Test model
# save the output from the neural network for given images.
output = model.predict(color_me)
#the values are between -1 and 1 so to restore the values of ab channels we need to multiply them by 128.
output = output * 128

In [12]:
#Save the output
from skimage import img_as_ubyte #Add a module that does float64 to uint8 conversion for us

# Output colorizations
for i in range(len(output)):
    # Create an empty matrix that has 3 channels, each 256 x 256 in preparation of the final rgb picture.
    create_image = np.zeros((256, 256, 3))
    # Fill the first layer with the lightness channel information
    create_image[:,:,0] = color_me[i][:,:,0]
    #fill the 2nd and 3rd channel with produced output.
    create_image[:,:,1:] = output[i]
    
    create_image = lab2rgb(create_image)
    create_image = img_as_ubyte(create_image) #convert float64 to uint8 to avoid lossy conversion.
    imsave("result/img_"+str(i)+".png", create_image)
    print("Saved picture number: ", i)

Saved picture number:  0
Saved picture number:  1
Saved picture number:  2
Saved picture number:  3
Saved picture number:  4
Saved picture number:  5
Saved picture number:  6
Saved picture number:  7
